In [12]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import stats
from scipy.sparse import issparse
import warnings
warnings.filterwarnings('ignore')

def h5ad_to_differential_expression_csv(h5ad_file_path, output_csv_path, 
                                      min_cells_per_cluster=10, 
                                      top_genes_per_cluster=100):
    """
    Convert h5ad file to differential expression CSV format.
    
    Parameters:
    h5ad_file_path: path to your h5ad file
    output_csv_path: path for output CSV file
    min_cells_per_cluster: minimum cells required for a cluster to be analyzed
    top_genes_per_cluster: number of top genes to include per cluster
    """
    
    # Load the h5ad file
    print("Loading h5ad file...")
    adata = sc.read_h5ad(h5ad_file_path)
    
    # Basic preprocessing if needed
    print("Preprocessing data...")
    
    
    # Basic filtering - remove genes expressed in very few cells
    sc.pp.filter_genes(adata, min_cells=3)
    
    # If data is not log-transformed, do basic normalization and log transformation
    if adata.X.max() > 20:  # Likely raw counts
        print("Normalizing and log-transforming data...")
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)
    
    # Perform clustering if no clusters exist
    if 'leiden' not in adata.obs.columns and 'louvain' not in adata.obs.columns:
        print("No clusters found. Performing clustering...")
        
        # Find highly variable genes
        sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
        adata.raw = adata
        adata = adata[:, adata.var.highly_variable]
        
        sc.tl.pca(adata, svd_solver='arpack', random_state=42)
        sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40, random_state=42)
        sc.tl.leiden(adata, resolution=0.5, random_state=42)
        
        # Use raw data for differential expression
        adata = adata.raw.to_adata()
        cluster_key = 'leiden'
    else:
        # Use existing clustering
        cluster_key = 'leiden' if 'leiden' in adata.obs.columns else 'louvain'
    
    print(f"Using cluster key: {cluster_key}")
    print(f"Found {len(adata.obs[cluster_key].unique())} clusters")
    
    # Perform differential expression analysis
    print("Performing differential expression analysis...")
    
    # Use scanpy's rank_genes_groups for differential expression
    sc.tl.rank_genes_groups(adata, cluster_key, method='wilcoxon', use_raw=False)
    
    # Extract results and format as requested
    print("Formatting results...")
    
    results = []
    clusters = adata.obs[cluster_key].unique()
    
    for cluster in clusters:
        # Skip clusters with too few cells
        cluster_size = (adata.obs[cluster_key] == cluster).sum()
        if cluster_size < min_cells_per_cluster:
            continue
            
        print(f"Processing cluster {cluster} ({cluster_size} cells)")
        
        # Get top genes for this cluster
        cluster_genes = adata.uns['rank_genes_groups']['names'][str(cluster)][:top_genes_per_cluster]
        cluster_scores = adata.uns['rank_genes_groups']['scores'][str(cluster)][:top_genes_per_cluster]
        cluster_pvals = adata.uns['rank_genes_groups']['pvals'][str(cluster)][:top_genes_per_cluster]
        cluster_pvals_adj = adata.uns['rank_genes_groups']['pvals_adj'][str(cluster)][:top_genes_per_cluster]
        cluster_logfoldchanges = adata.uns['rank_genes_groups']['logfoldchanges'][str(cluster)][:top_genes_per_cluster]
        
        # Calculate additional statistics
        cluster_mask = adata.obs[cluster_key] == cluster
        other_mask = adata.obs[cluster_key] != cluster
        
        for i, gene in enumerate(cluster_genes):
            if pd.isna(gene):
                continue
                
            try:
                gene_idx = adata.var_names.get_loc(gene)
            except KeyError:
                continue
            
            # Get expression data for this gene
            if issparse(adata.X):
                gene_data = adata.X[:, gene_idx].toarray().flatten()
            else:
                gene_data = adata.X[:, gene_idx]
            
            # Calculate percentage of cells expressing the gene in cluster vs others
            cluster_expr = gene_data[cluster_mask]
            other_expr = gene_data[other_mask]
            
            pct_1 = np.mean(cluster_expr > 0)  # Percentage expressing in cluster
            pct_2 = np.mean(other_expr > 0)    # Percentage expressing in other clusters
            
            # Get statistics
            p_val = cluster_pvals[i] if not pd.isna(cluster_pvals[i]) else 0
            p_val_adj = cluster_pvals_adj[i] if not pd.isna(cluster_pvals_adj[i]) else 0
            avg_log2FC = cluster_logfoldchanges[i] if not pd.isna(cluster_logfoldchanges[i]) else 0
            
            # Format the result
            result = {
                '': gene,
                'p_val': p_val,
                'avg_log2FC': avg_log2FC,
                'pct.1': round(pct_1, 3),
                'pct.2': round(pct_2, 3), 
                'p_val_adj': p_val_adj,
                'cluster': f'cluster_{cluster}',  # You can modify this naming
                'gene': gene
            }
            
            results.append(result)
    
    # Convert to DataFrame and save
    print("Saving results...")
    df = pd.DataFrame(results)
    
    # Sort by cluster and then by avg_log2FC
    df = df.sort_values(['cluster', 'avg_log2FC'], ascending=[True, False])
    
    # Save to CSV
    df.to_csv(output_csv_path, index=False)
    
    print(f"Results saved to {output_csv_path}")
    print(f"Total entries: {len(df)}")
    print(f"Clusters processed: {df['cluster'].nunique()}")
    
    return df, adata

In [ ]:
import pandas as pd

# Load the labelled CSV
labelled_df = pd.read_csv("FastAnalysisResults_summary.csv")

# Remove 'cluster_' prefix so keys match adata.obs[cluster_key] values
labelled_df["cluster"] = labelled_df["cluster"].astype(str).str.replace("cluster_", "")

# Build mapping from cluster number (as string) to predicted sub cell type
cluster_to_type = labelled_df.drop_duplicates("cluster").set_index("cluster")["Predicted Main Cell Type"].to_dict()

# Use the AnnData object you already loaded (adata)
cluster_key = "leiden" if "leiden" in adata.obs.columns else "louvain"

# Ensure both mapping keys and adata.obs values are strings
adata.obs["predicted_sub_cell_type"] = adata.obs[cluster_key].astype(str).map(
    lambda c: cluster_to_type.get(str(c), "Unknown")
)

# Map predicted sub cell types to each cell based on cluster (convert to string for matching)
adata.obs["predicted_sub_cell_type"] = adata.obs[cluster_key].astype(str).map(lambda c: cluster_to_type.get(c, "Unknown"))


adata.obs['malignancy'] = adata.obs['predicted_sub_cell_type'].str.lower().apply(
    lambda x: 'malignant' if ('cancer' in x or 'tumor' in x) else 'non-malignant'
)


# Save the annotated AnnData object
adata.write("dataset_labelled_cassia.h5ad")

print("Annotated h5ad file saved as dataset_labelled_cassia.h5ad")

Annotated h5ad file saved as dataset_with_predicted_types.h5ad


In [50]:
adata.obs.head()

,leiden,predicted_sub_cell_type,malignancy
Gao2021_AAACCTGCAGTGACAG,37,"luminal epithelial cells, basal epithelial cel...",malignant
Gao2021_AAACCTGGTCGAGATG,37,"luminal epithelial cells, basal epithelial cel...",malignant
Gao2021_AAACCTGTCACCGGGT,46,"cancer-associated fibroblasts (CAFs), myofibro...",malignant
Gao2021_AAACGGGGTGCACTTA,37,"luminal epithelial cells, basal epithelial cel...",malignant
Gao2021_AAACGGGTCACGGTTA,37,"luminal epithelial cells, basal epithelial cel...",malignant
